In [ ]:
import os
import warnings

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from cartopy.io import DownloadWarning as CartopyDownloadWarning
from ipywidgets import interact

from canopy_app import config_cases, run, run_config_sens, DEFAULT_POINT_INPUT, REPO

warnings.filterwarnings("ignore", category=CartopyDownloadWarning)

xr.set_options(display_expand_data=False, display_expand_attrs=False)

plt.rcParams.update(
    {
        "figure.autolayout": True,
        "axes.formatter.limits": (-3, 4),
        "axes.formatter.use_mathtext": True,
        "figure.max_open_warning": 0,
    }
)

%matplotlib widget

## Default case (2-D)

Southeast US

```
provide spatial plots for 3D vars (e.g., ave. in-canopy ws, kz, rjcf and sum in-canopy biogenic isoprene) and 2D vars (flame height, waf)
```

In [ ]:
inp = xr.open_dataset("../input/gfs.t12z.20220701.sfcf000.canopy.nc")

In [ ]:
inp

In [ ]:
%%time

ds = run()

In [ ]:
proj = ccrs.Mercator()
tran = ccrs.PlateCarree()

fig = plt.figure(constrained_layout=True)
ax = fig.add_subplot(projection=proj)

ax.add_feature(cfeature.STATES, edgecolor="orangered", linewidth=1)
ax.coastlines(color="orangered", linewidth=1.5)
ax.gridlines(draw_labels=True)

ds.waf.plot(
    x="lon", y="lat", ax=ax, transform=tran, cbar_kwargs=dict(orientation="horizontal")
)

In [ ]:
proj = ccrs.Mercator()
tran = ccrs.PlateCarree()

# NOTE: currently flame height and wind speed var names are different in nc vs txt output ds
vns = ["canwind", "kz", "rjcf", "emi_isop", "flameh", "waf"]

fig, axs = plt.subplots(
    2,
    3,
    subplot_kw=dict(projection=proj),
    constrained_layout=True,
    figsize=(12, 7),
    sharex=True,
    sharey=True,
)


ds["hc"] = (("y", "x"), inp.fh.squeeze().values)
hc = ds.hc
dz = ds.z.diff("z")
for i, (vn, ax) in enumerate(zip(vns, axs.flat)):
    ax.add_feature(cfeature.STATES, edgecolor="orangered", linewidth=1)
    ax.coastlines(color="orangered", linewidth=1.5)
    gl = ax.gridlines(draw_labels=True)
    if i % ax.get_gridspec().ncols in range(1, ax.get_gridspec().ncols - 1):
        gl.right_labels = False
        gl.left_labels = False

    da = ds[vn]
    if vn in {"canwind", "kz", "rjcf"}:
        # Canopy mean
        da = da.where(da.z <= hc).mean("z")
        agg_label = "canopy mean"
    elif vn in {"emi_isop"}:
        # Integrate
        da = (da * dz).sum("z")
        agg_label = "integral"
        assert ds[vn].attrs["units"] == "kg m-3 s-1"
        da.attrs["units"] = "kg m-2 s-1"
    else:
        assert da.dims == ("y", "x"), "2-D"
        agg_label = None

    cbar_label = vn
    if "units" in da.attrs:
        cbar_label += f" [{da.units}]"
    if agg_label is not None:
        cbar_label += f"\n({agg_label})"
    da.plot(
        x="lon",
        y="lat",
        ax=ax,
        transform=tran,
        cbar_kwargs=dict(orientation="horizontal", label=cbar_label),
    )
    ax.set_title("")

## Canopy thresholds

```
Maybe, lai_thresh = 0.1
frt_thresh = 0.1
fch_thresh = 0.5...
and
lai_thresh = 0.5
frt_thresh = 0.5
fch_thresh = 0.5
or,
lai_thresh = 0.1
frt_thresh = 0.5
fch_thresh = 10.0 (or 3.0) or something only representing tall forest stands...
```

In [ ]:
%%time

threshes = [
    # lai, frt, fch
    (0.1, 0.1, 0.5),  # defaults
    (0.5, 0.5, 0.5),
    (0.1, 0.5, 3),
    (0.1, 0.5, 10),
]

lai_threshes, frt_threshes, fch_threshes = [list(x) for x in zip(*threshes)]

cases = config_cases(
    lai_thresh=lai_threshes,
    frt_thresh=frt_threshes,
    fch_thresh=fch_threshes,
)
print(cases)

ds = run_config_sens(cases)

In [ ]:
with mpl.rc_context({"figure.autolayout": False}):
    axs = ds.waf.plot(x="lon", y="lat", col="case", col_wrap=2, add_colorbar=False)

In [ ]:
fig = plt.figure(
    constrained_layout=True,
    figsize=(9, 7.2),
)


@interact(
    vn=["waf", "canwind", "kz", "rjcf", "emi_isop", "flameh"],
)
def f(vn):
    fig.clf()
    axs = []
    share = None
    for i in range(2 * 2):
        share = ax = fig.add_subplot(
            2,
            2,
            i + 1,
            projection=proj,
            sharex=share,
            sharey=share,
        )
        axs.append(ax)

    da = ds[vn]
    if vn in {"canwind", "kz", "rjcf"}:
        # Canopy mean
        da = da.where(da.z <= hc).mean("z")
        agg_label = "canopy mean"
    elif vn in {"emi_isop"}:
        # Integrate
        da = (da * dz).sum("z")
        agg_label = "integral"
        assert ds[vn].attrs["units"] == "kg m-3 s-1"
        da.attrs["units"] = "kg m-2 s-1"
    else:
        assert da.isel(case=0).dims == ("y", "x"), "2-D"
        agg_label = None

    dq = 0.05
    vmin, vmax = da.quantile([dq, 1 - dq]).values

    cbar_label = vn
    if "units" in da.attrs:
        cbar_label += f" [{da.units}]"
    if agg_label is not None:
        cbar_label += f"\n({agg_label})"

    for i, (case, ax) in enumerate(zip(cases, axs)):
        ax.add_feature(cfeature.STATES, edgecolor="orangered", linewidth=1)
        ax.coastlines(color="orangered", linewidth=1.5)
        gl = ax.gridlines(draw_labels=True)
        if i % 2 == 1:
            gl.right_labels = False
            gl.left_labels = False

        im = da.isel(case=i).plot.pcolormesh(
            x="lon",
            y="lat",
            transform=tran,
            vmin=vmin,
            vmax=vmax,
            add_colorbar=False,
            ax=ax,
        )

        ax.set_title(
            ", ".join(f"{k}={v}" for k, v in case["userdefs"].items()),
            color="forestgreen",
            size=11,
        )

    fig.colorbar(im, ax=axs, orientation="horizontal", shrink=0.7, label=cbar_label)

## PAI options

```
For, flameh_opt/flameh_set,/frp_fac, here is a suggestion of case sensitivities:
0, 1.0, 1.0  (Straight FRP to flameh calculation, only for active FRP/fire points, no FRP tuning)
1, 2.0, 1.0  (User-set flame height = 2.0 m for all canopy points)
2, 2.0, 1.0  (FRP to flameh calculation for active FRP/fire points,   user-set flame height = 2.0 m elsewhere, no FRP tuning)
3, 0.5, 1.0  (Flameh is set to 0.5*Hc (m) for all canopy points)
4, 0.5, 1.0  (FRP to flameh calculation for active FRP/fire points, flameh is set to 0.5*Hc (m) elsewhere, no FRP tuning)
5, 0.5, 10.0  (FRP to flameh calculation for active FRP/fire points with crowning dependence (flameh=Hc), flameh is set to 0.5*Hc (m) elsewhere, FRP increased by a factor of 10)
```

## Flame height options

```
For pai_opt/pai_set, here is a list of sensitivities:
0, 1 (PAI from Katul et al., 2004 vegetation types/lookup table)
2, 1 (PAI estimated from model LAI)
3, 1 (User-set PAI = 1)
3, 2 (User-set PAI = 2)
3, 4 (User-set PAI = 4)
```

## Canopy environment coefficient (biogenics)

```
Also, we should have a sensitivity for the biogenics, using the bio_cce (canopy environment coefficient) that is a tuning parameter for different model inputs..(the default 0.21 is based on Silva et al. and GEOS-Chem environment)...
```

## Default point case

In [ ]:
%%time

ds = run(
    config={
        "filenames": {"file_vars": "../input/input_variables_point.txt"},
        "userdefs": {"infmt_opt": 1, "nlat": 1, "nlon": 1},
    },
)

In [ ]:
vns = ["ws", "kz", "rjcf", "emi_isop"]

fig, axs = plt.subplots(1, len(vns), figsize=(10, 4), sharey=True)

for vn, ax in zip(vns, axs.flat):
    ax.axhline(ds.canopy_height, c="0.5", ls="--")
    ds[vn].plot(y="z", ax=ax)

ax.set_ylim(ymin=0);

In [ ]:
(
    ds[[vn for vn in ds.data_vars if vn.startswith("emi_")]]
    .sel(z=10, method="nearest")
    .to_pandas()
    .to_frame()
    .plot.bar(figsize=(6, 3), legend=False)
)
plt.yscale("log")

## $z_0 / h_c$

The namelist parameter `z0ghc` represents the ratio of ground roughness length to canopy top height, i.e.
$z_0 / h_c$. In general, they increase together, but this is still a tunable parameter.

In [ ]:
%%time

cases = config_cases(
    file_vars="../input/input_variables_point.txt",
    infmt_opt=1,
    nlat=1,
    nlon=1,
    z0ghc=np.power(10, np.linspace(-4, np.log10(0.25), 100)).tolist(),
    product=False,
)
ds = run_config_sens(cases)
ds

In [ ]:
hc = ds.canopy_height.isel(case=0)
assert (ds.canopy_height == hc).all()
ds_c = ds.isel(z=ds.z <= hc).mean("z")  # canopy mean

vns = [
    "ws",
    "kz",
    "waf",
]

fig, axs = plt.subplots(1, len(vns), figsize=(9, 3.5), sharey=False)

fig.suptitle("Canopy mean")

for vn, ax in zip(vns, axs.flat):
    ax.plot(ds_c["z0ghc"], ds_c[vn])
    ax.set(ylabel=vn, xlabel="$z_0 / h_c$")
    ax.set_xscale("log")

## Input variables

We can create different point input files based on the default case and run the model for each,
allowing us to examine the sensitivity to the input variables.

Note that files matching `test_*.txt` are gitignored.

In [ ]:
DEFAULT_POINT_INPUT.T.rename(columns={0: "value"})

In [ ]:
%%time

# Create input files
clu = np.linspace(0.3, 1.0, 50)
df = DEFAULT_POINT_INPUT.copy()
paths = []
for i, clu_i in enumerate(clu):
    df["clu"] = clu_i
    p = REPO / "input" / f"test_point_{i:02}.txt"
    paths.append(p)
    df.to_csv(p, index=False)

# Run cases
cases = config_cases(
    file_vars=[p.as_posix() for p in paths],
    infmt_opt=1,
    nlat=1,
    nlon=1,
    product=False,
)
ds = run_config_sens(cases)

# Clean up input files
for p in paths:
    os.remove(p)

In [ ]:
hc = ds.canopy_height.isel(case=0)
assert (ds.canopy_height == hc).all()
ds_c = ds.isel(z=ds.z <= hc).mean("z")  # canopy mean

vns = [
    "rjcf",
    "emi_isop",
    "emi_apin",
]

fig, axs = plt.subplots(1, len(vns), figsize=(9, 3.5), sharey=False)

fig.suptitle("Canopy mean")

for vn, ax in zip(vns, axs.flat):
    ax.plot(clu, ds_c[vn])
    ax.set(ylabel=vn, xlabel="CLU")